In [25]:
import librosa as ls
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [26]:
### Audio pre-processing 
def normalize(X):
    temp = X-np.mean(X,axis=0)
    Y = temp/np.std(X,axis=0)
    return Y

chi_mfcc = []
bhi_mfcc = []
for i in os.listdir('./chi'):
    if i.startswith('chi'):
        path = os.path.join('./chi',i)
        wav,sr = ls.load(path, duration=0.21 ,sr=None)
#         print(ls.get_duration(wav))
        mfccs = ls.feature.mfcc(y=wav, sr=sr, n_mfcc=13, hop_length=int(0.015*sr), n_fft=int(0.025*sr))
        chi_mfcc.append(mfccs.T)

for i in os.listdir('./bhi'):
    if i.startswith('bhi'):
        path = os.path.join('./bhi',i)
        wav,sr = ls.load(path, duration=0.21, sr=None)
        mfccs = ls.feature.mfcc(y=wav, sr=sr, n_mfcc=13, hop_length=int(0.015*sr), n_fft=int(0.025*sr))
        bhi_mfcc.append(mfccs.T)
        
# Train-test split. Sklearn modules have been used for simplicity.
chi_mfcc = normalize(np.array(chi_mfcc))
bhi_mfcc = normalize(np.array(bhi_mfcc))

chi_train,chi_test = train_test_split(chi_mfcc, test_size=0.2)
bhi_train,bhi_test = train_test_split(bhi_mfcc, test_size=0.2)
print(chi_test.shape, bhi_test.shape)

(8, 15, 13) (8, 15, 13)


/home/legion/.local/lib/python3.7/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


In [27]:
N,N1 = chi_train[0].shape # number of nodes, parameters of each gaussian in the node
k = 7                     # model order
M  = np.random.uniform(-1,1,(k,N1))   # mean initialization
C = []
for i in range(k):
    C.append(np.identity(N1))
C = np.array(C)
A = np.ones((k,k))*1.0/3 # transition probabilities
P = np.random.uniform(1,10,(1,k))
P = P/np.sum(P)

In [28]:
def normal(x,Mean,covar):
    d=len(x)
    x=x.reshape(1,len(Mean))
    Mean=Mean.reshape(1,len(Mean))
    inverse=np.linalg.inv(covar)
    det=np.linalg.det(covar)
    nr=np.exp(-0.5*np.matmul(np.matmul(x-Mean,inverse),(x-Mean).T))
    dr=np.sqrt(det*((2*np.pi)**d))
    tmp=np.asscalar(nr/dr)
    return tmp

In [29]:
def alpha(x,pis,weights,mean,covar):
    alphas = []
    N,_ = x.shape
    i=0
    for i in range(N):
        probs = []
        for j in range(k):
            probs.append(normal(x[i],mean[j],covar[j]))
        probs = np.array(probs)
        probs = probs.reshape(1,k)
        if(i==0):
            alphas.append(pis*probs)
        else:
            alphas.append(probs*np.matmul(alphas[-1],weights))
    alphas = np.array(alphas)
    return alphas

In [30]:
def beta(x,weights,mean,covar):
    betas = []
    N,_ = x.shape
    i = N-1
    while(i>=0):
        probs = []
        for j in range(k):
            probs.append(normal(x[i],mean[j],covar[j]))
        probs = np.array(probs)
        probs = probs.reshape(k,1)
        if(i==N-1):
            temp = np.ones((k,1))
            betas.append(temp)
        else:
            betas.append(np.matmul(weights,betas[-1]*probs))
        i=i-1
    betas = np.array(betas)
    betas = betas[::-1]
    return betas

In [31]:
def gamma(alphas,betas):
    gammas = []
    for i in range(0,len(alphas)):
        gammas.append((alphas[i]*betas[i].reshape(1,k))/np.sum(alphas[-1]))
    gammas = np.array(gammas)
    return gammas

In [32]:
def epsilon(x,alphas,betas,weights,mean,covar):
    epsilons = []
    for i in range(1,len(alphas)):
        j=0
        prob = []
        for j in range(k):
            prob.append(normal(x[i],mean[j],covar[j]))
        prob = np.array(prob)
        prob = prob.reshape(1,k)
#         t = alphas[i-1]*prob*(np.matmul(weights,betas[i]))/np.sum(alphas[-1])
        t = np.matmul(np.matmul(weights,alphas[i-1].T),(prob*betas[i].reshape(prob.shape)))/np.sum(alphas[-1])
        epsilons.append(t)
    epsilons = np.array(epsilons)
    return epsilons,prob

In [33]:
count = 0
while(count < 300):
    Q = 0
    P_temp=A_temp=C_temp=M_temp=0
    Len = len(chi_train)
    for l in range(Len):
        alphas = alpha(chi_train[l],P,A,M,C)
        betas = beta(chi_train[l],A,M,C)
        gammas = gamma(alphas,betas)
        epsilons,probs_chi = epsilon(chi_train[l],alphas,betas,A,M,C)
        P_new = gammas[0]/np.sum(gammas[0])
        gammas=gammas.reshape(N,k)
        M_new = np.matmul(chi_train[l].T,gammas)/np.sum(gammas,axis=0)
        M_new = M_new.T
        A_new = np.sum(epsilons,axis=0)/np.sum(np.sum(epsilons,axis=0),axis=1)
        covar_new=[]
        temp1 = gammas.T
        for i in range(k):
            covar1 = np.matmul((chi_train[l]-M[i]).T,(chi_train[l]-M[i]))
            covar11 = np.zeros(covar1.shape)
            for j in range(0,len(temp1[i])):
                covar11 = covar11 + covar1*temp1[i][j]
            covar11 = covar11/np.sum(temp1[i])
            covar_new.append(covar11)
        covar_new = np.array(covar_new)
        probs_chi = np.array(probs_chi)
        P_temp += P_new
        A_temp += A_new
        C_temp += covar_new
        M_temp += M_new
        Q += np.matmul(gammas[0],np.log(P).T) + np.sum(epsilons*np.log(A)) + np.sum(gammas*np.log(probs_chi))
    P = P_temp/Len
    A = A_temp/Len
    C = C_temp/Len
    M = M_temp/Len
    Q = Q/Len    
    print(Q)
    count+=1

P_chi = P
A_chi = A
C_chi = C
M_chi = M

[-13301.581021]
[-707.19279124]
[-688.59570332]
[-683.53908909]
[-683.24602139]
[-683.24436643]
[-683.24380034]
[-683.24370034]
[-683.24359834]
[-683.24349612]
[-683.24339384]
[-683.24329148]
[-683.24318903]
[-683.2430865]
[-683.24298389]
[-683.24288118]
[-683.24277839]
[-683.24267552]
[-683.24257255]
[-683.24246951]
[-683.24236637]
[-683.24226315]
[-683.24215984]
[-683.24205644]
[-683.24195296]
[-683.24184939]
[-683.24174574]
[-683.241642]
[-683.24153817]
[-683.24143425]
[-683.24133025]
[-683.24122616]
[-683.24112198]
[-683.24101772]
[-683.24091337]
[-683.24080893]
[-683.24070441]
[-683.2405998]
[-683.2404951]
[-683.24039031]
[-683.24028544]
[-683.24018048]
[-683.24007543]
[-683.23997029]
[-683.23986507]
[-683.23975976]
[-683.23965436]
[-683.23954888]
[-683.2394433]
[-683.23933764]
[-683.23923189]
[-683.23912606]
[-683.23902013]
[-683.23891412]
[-683.23880802]
[-683.23870184]
[-683.23859556]
[-683.2384892]
[-683.23838275]
[-683.23827621]
[-683.23816959]
[-683.23806287]
[-683.23795607]

In [34]:
count = 0
while(count < 300):
    Q = 0
    P_temp=A_temp=C_temp=M_temp=0
    Len = len(chi_train)
    for l in range(Len):
        alphas = alpha(bhi_train[l],P,A,M,C)
        betas = beta(bhi_train[l],A,M,C)
        gammas = gamma(alphas,betas)
        epsilons,probs_bhi = epsilon(bhi_train[l],alphas,betas,A,M,C)
        P_new = gammas[0]/np.sum(gammas[0])
        gammas=gammas.reshape(N,k)
        M_new = np.matmul(bhi_train[l].T,gammas)/np.sum(gammas,axis=0)
        M_new = M_new.T
        A_new = np.sum(epsilons,axis=0)/np.sum(np.sum(epsilons,axis=0),axis=1)
        covar_new=[]
        temp1 = gammas.T
        for i in range(k):
            covar1 = np.matmul((bhi_train[l]-M[i]).T,(bhi_train[l]-M[i]))
            covar11 = np.zeros(covar1.shape)
            for j in range(0,len(temp1[i])):
                covar11 = covar11 + covar1*temp1[i][j]
            covar11 = covar11/np.sum(temp1[i])
            covar_new.append(covar11)
        covar_new = np.array(covar_new)
        probs_bhi = np.array(probs_bhi)
        P_temp += P_new
        A_temp += A_new
        C_temp += covar_new
        M_temp += M_new
        Q += np.matmul(gammas[0],np.log(P).T) + np.sum(epsilons*np.log(A)) + np.sum(gammas*np.log(probs_bhi))
    P = P_temp/Len
    A = A_temp/Len
    C = C_temp/Len
    M = M_temp/Len
    Q = Q/Len
    print(Q)
    count+=1

P_bhi = P
A_bhi = A
C_bhi = C
M_bhi = M

[-648.42876114]
[-699.87369679]
[-700.0726603]
[-700.0584147]
[-700.05687106]
[-700.05680695]
[-700.0568268]
[-700.05685037]
[-700.05687429]
[-700.05689823]
[-700.05692216]
[-700.05694607]
[-700.05696998]
[-700.05699387]
[-700.05701775]
[-700.05704162]
[-700.05706547]
[-700.05708931]
[-700.05711314]
[-700.05713696]
[-700.05716076]
[-700.05718456]
[-700.05720834]
[-700.0572321]
[-700.05725586]
[-700.0572796]
[-700.05730333]
[-700.05732705]
[-700.05735075]
[-700.05737444]
[-700.05739812]
[-700.05742179]
[-700.05744545]
[-700.05746909]
[-700.05749272]
[-700.05751634]
[-700.05753994]
[-700.05756354]
[-700.05758712]
[-700.05761069]
[-700.05763424]
[-700.05765779]
[-700.05768132]
[-700.05770484]
[-700.05772835]
[-700.05775184]
[-700.05777533]
[-700.0577988]
[-700.05782226]
[-700.0578457]
[-700.05786914]
[-700.05789256]
[-700.05791597]
[-700.05793937]
[-700.05796275]
[-700.05798613]
[-700.05800949]
[-700.05803284]
[-700.05805618]
[-700.0580795]
[-700.05810281]
[-700.05812612]
[-700.05814941]


KeyboardInterrupt: 

In [35]:
Len = len(chi_test)
count = 0
chip = []
bhip = []
for l in range(Len):
    alphas = alpha(bhi_test[l],P,A_chi,M_chi,C_chi)
    betas = beta(bhi_test[l],A_chi,M_chi,C_chi)
    gammas = gamma(alphas,betas)
    epsilons,probs_bhi = epsilon(bhi_test[l],alphas,betas,A_chi,M_chi,C_chi)
    px = np.sum(np.matmul(alphas,betas))
    bhip.append(px)
    alphas = alpha(chi_test[l],P,A_chi,M_chi,C_chi)
    betas = beta(chi_test[l],A_chi,M_chi,C_chi)
    gammas = gamma(alphas,betas)
    epsilons,probs_chi = epsilon(chi_test[l],alphas,betas,A_chi,M_chi,C_chi)
    px = np.sum(np.matmul(alphas,betas))
    chip.append(px)

for i in range(Len):
    if(chip[i]>bhip[i]):
        count += 1
print(count/8.0)

0.75


The model has been trained with the hidden nodes having a degree of 5.

Note that for training the HMM for bhi, chi's P,A,M,C have been used as initializations.

The accuracies are as follows:
1. degree 3 - 0.625 (epochs = 200 per HMM)
2. degree 5 - 0.875 (epochs = 200 per HMM)
3. degree 7 - 0.75  (lesser epochs than previous case due to time constraint)